In [1]:
import category_encoders
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.pipeline import Pipeline
from sklearn import set_config
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

**AQUÍ SOLO SE MUESTRAN TRATAMIENTOS QUE HAY QUE HACERLE A LOS DATOS CON SUS RESPECTIVOS RESULTADOS. NO OBSTANTE, LUEGO HAY QUE METER ESTO EN UN COLUMN TRANSFORMER Y DESPUÉS EN UNA PIPELINE.**

In [2]:
#Primero, leemos el archivo csv. Está separado por puntos y comas.

datosCrudos=pd.read_csv('pokemon-data.csv', delimiter=";")
datosCrudos

,Name,Types,Abilities,Tier,HP,Attack,Defense,Special Attack,Special Defense,Speed,Next Evolution(s),Moves
0,Abomasnow,"['Grass', 'Ice']","['Snow Warning', 'Soundproof']",PU,90,92,75,92,85,60,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le..."
1,Abomasnow-Mega,"['Grass', 'Ice']",['Snow Warning'],NUBL,90,132,105,132,105,30,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le..."
2,Abra,[‘Psychic’],"['Inner Focus', 'Magic Guard', 'Synchronize']",LC,25,20,15,105,55,90,['Kadabra'],"['Teleport', 'Ally Switch', 'Barrier', 'Encore..."
3,Absol,['Dark'],"['Justified', 'Pressure', 'Super Luck']",PU,65,130,60,75,60,75,[],"['Perish Song', 'Future Sight', 'Me First', 'R..."
4,Absol-Mega,['Dark'],['Magic Bounce'],RUBL,65,150,60,115,60,115,[],"['Perish Song', 'Future Sight', 'Me First', 'R..."
...,...,...,...,...,...,...,...,...,...,...,...,...
913,Zubat,"['Flying', 'Poison']","['Infiltrator', 'Inner Focus']",LC,40,45,35,30,40,55,['Golbat'],"['Absorb', 'Leech Life', 'Supersonic', 'Astoni..."
914,Zweilous,"['Dark', 'Dragon']",['Hustle'],NaN,72,85,70,65,70,58,['Hydreigon'],"['Double Hit', 'Dragon Rage', 'Focus Energy', ..."
915,Zygarde,"['Dragon', 'Ground']","['Aura Break', 'Power Construct']",OU,108,100,121,81,95,95,"['Zygarde-10%', 'Zygarde-Complete']","['Glare', 'Bulldoze', 'Dragon Breath', 'Bite',..."
916,Zygarde-10%,"['Dragon', 'Ground']","['Aura Break', 'Power Construct']",RU,54,100,71,61,85,115,[],"['Glare', 'Bulldoze', 'Dragon Breath', 'Bite',..."


In [3]:
#Vamos a modificar el índice de las filas
datosCrudos.index = datosCrudos['Name']

Nos interesa desdoblar las columnas de los tipos, ya que hay muchas combinaciones de tipos diferentes. Al desdoblar la columna de tipos, obtenemos dos variables que sólo pueden tener 18 categorías (tipo 2 19 categorías), lo cual nos va a dar más información sobre lo bueno que es cada tipo a la hora
de  determinar el potencial competitivo de un Pokémon. Para ello, creamos una clase `TypeCategorizer()`que se va a encargar de esto. Tiene los métodos `fit` y `transform`para poder utilziarse en una *Pipeline*.

In [4]:
class TypeCategorizer():
    # Constructor de la clase
    def __init__(self, columns=None): #Si recibimos algo
        self.columns = columns

    #Fit
    def fit(self, X):
        X=pd.DataFrame(X) #Por si recibiéramos un array de numpy
        if self.columns == None:
            self.columns=X.columns
        return self

    #Transform
    def transform(self, X):
        #Creamos una copia de X para no perder los datos originales
        Xaux=X.copy()
        #Accedemos a la columna de tipos, y rellenamos las columnas acorde
        for c in self.columns:
            caracteresAEliminar = ["[", "]", "'"]
            for char in caracteresAEliminar:
                Xaux.loc[:,c]= Xaux.loc[:,c].str.replace(char,'')
            #Después, para los Pokémon que solo tienen un tipo, vamos a añadirles un segundo tipo "nulo"
            Xaux[['Type 1', 'Type 2']] = Xaux.loc[:,c].str.split(',', expand=True) #Expand true para que cree dos nuevas columnas

            #Por último, hay que borrar la columna de "Types" original
            Xaux=Xaux.drop(["Types"], axis=1)
        return Xaux

In [5]:
categorizadorDeTipos = TypeCategorizer(columns = ["Types"])
categorizadorDeTipos.fit(datosCrudos)
datosConTipos = categorizadorDeTipos.transform(datosCrudos)
datosConTipos

,Name,Abilities,Tier,HP,Attack,Defense,Special Attack,Special Defense,Speed,Next Evolution(s),Moves,Type 1,Type 2
Name,,,,,,,,,,,,,
Abomasnow,Abomasnow,"['Snow Warning', 'Soundproof']",PU,90,92,75,92,85,60,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le...",Grass,Ice
Abomasnow-Mega,Abomasnow-Mega,['Snow Warning'],NUBL,90,132,105,132,105,30,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le...",Grass,Ice
Abra,Abra,"['Inner Focus', 'Magic Guard', 'Synchronize']",LC,25,20,15,105,55,90,['Kadabra'],"['Teleport', 'Ally Switch', 'Barrier', 'Encore...",‘Psychic’,None
Absol,Absol,"['Justified', 'Pressure', 'Super Luck']",PU,65,130,60,75,60,75,[],"['Perish Song', 'Future Sight', 'Me First', 'R...",Dark,None
Absol-Mega,Absol-Mega,['Magic Bounce'],RUBL,65,150,60,115,60,115,[],"['Perish Song', 'Future Sight', 'Me First', 'R...",Dark,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zubat,Zubat,"['Infiltrator', 'Inner Focus']",LC,40,45,35,30,40,55,['Golbat'],"['Absorb', 'Leech Life', 'Supersonic', 'Astoni...",Flying,Poison
Zweilous,Zweilous,['Hustle'],NaN,72,85,70,65,70,58,['Hydreigon'],"['Double Hit', 'Dragon Rage', 'Focus Energy', ...",Dark,Dragon
Zygarde,Zygarde,"['Aura Break', 'Power Construct']",OU,108,100,121,81,95,95,"['Zygarde-10%', 'Zygarde-Complete']","['Glare', 'Bulldoze', 'Dragon Breath', 'Bite',...",Dragon,Ground


Vamos a realizar el mismo tratamiento para las habilidades. Es decir, vamos a generar tres columnas, una por cada habilidad (un Pokémon puede tener 1,2 ó 3 habilidades de las que escoger).

In [6]:
def countCaracter(text, character):
    apariciones=0
    for char in text:
        if char==character:
            apariciones+=1
    return apariciones

class AbilityCategorizer():
    # Constructor de la clase
    def __init__(self, columns=None): #Si recibimos algo
        self.columns = columns

    #Fit
    def fit(self, X):
        X=pd.DataFrame(X) #Por si recibiéramos un array de numpy
        if self.columns == None:
            self.columns=X.columns
        return self

    #Transform
    def transform(self, X):
        #Creamos una copia de X para no perder los datos originales
        Xaux=X.copy()
        #Accedemos a la columna de habilidades, y modificamos las columnas acorde
        for c in self.columns:
            caracteresAEliminar = ["[", "]", "'"]
            for char in caracteresAEliminar:
                Xaux.loc[:,c]= Xaux.loc[:,c].str.replace(char,'')
            #Para los Pokémon que tienen sólo una habilidad, vamos a añadirles dos habilidades nulas; y para los
            #que tengan 2 habilidades, les añadimos una extra.
            for poke in Xaux.loc[:,c].index:
                if countCaracter(Xaux.loc[poke,c], ",")==0:
                    Xaux.loc[poke,c]=Xaux.loc[poke,c]+", None, None"
                elif countCaracter(Xaux.loc[poke,c], ",")==1:
                    Xaux.loc[poke,c]=Xaux.loc[poke,c]+", None"
            #Creamos tres columnas nuevas
            Xaux[['Ability 1', 'Ability 2', 'Ability 3']] = Xaux.loc[:,c].str.split(',', expand=True) #Expand true para que cree dos nuevas columnas
            #Por último, hay que borrar la columna de "Abilities" original
            Xaux=Xaux.drop(["Abilities"], axis=1)
        return Xaux

In [7]:
categorizadorDeHabilidades = AbilityCategorizer(columns = ["Abilities"])
categorizadorDeHabilidades.fit(datosConTipos)
datosConTiposHabilidades = categorizadorDeHabilidades.transform(datosConTipos)
datosConTiposHabilidades

,Name,Tier,HP,Attack,Defense,Special Attack,Special Defense,Speed,Next Evolution(s),Moves,Type 1,Type 2,Ability 1,Ability 2,Ability 3
Name,,,,,,,,,,,,,,,
Abomasnow,Abomasnow,PU,90,92,75,92,85,60,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le...",Grass,Ice,Snow Warning,Soundproof,None
Abomasnow-Mega,Abomasnow-Mega,NUBL,90,132,105,132,105,30,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le...",Grass,Ice,Snow Warning,None,None
Abra,Abra,LC,25,20,15,105,55,90,['Kadabra'],"['Teleport', 'Ally Switch', 'Barrier', 'Encore...",‘Psychic’,None,Inner Focus,Magic Guard,Synchronize
Absol,Absol,PU,65,130,60,75,60,75,[],"['Perish Song', 'Future Sight', 'Me First', 'R...",Dark,None,Justified,Pressure,Super Luck
Absol-Mega,Absol-Mega,RUBL,65,150,60,115,60,115,[],"['Perish Song', 'Future Sight', 'Me First', 'R...",Dark,None,Magic Bounce,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zubat,Zubat,LC,40,45,35,30,40,55,['Golbat'],"['Absorb', 'Leech Life', 'Supersonic', 'Astoni...",Flying,Poison,Infiltrator,Inner Focus,None
Zweilous,Zweilous,NaN,72,85,70,65,70,58,['Hydreigon'],"['Double Hit', 'Dragon Rage', 'Focus Energy', ...",Dark,Dragon,Hustle,None,None
Zygarde,Zygarde,OU,108,100,121,81,95,95,"['Zygarde-10%', 'Zygarde-Complete']","['Glare', 'Bulldoze', 'Dragon Breath', 'Bite',...",Dragon,Ground,Aura Break,Power Construct,None


Los nombres de los Pokémon no tienen relevancia competitiva, por lo que simplemente vamos a aplicar una codificación ordinal sobre esta columnna. Previsiblemente, al realizar la selección de variables, esta variable no será escogida, pero de igual forma necesitamos codificarla.

In [8]:
encodificadorOrdinal = ce.OrdinalEncoder(cols=["Name"])
#La entrenamos con el DataFrame sin datos faltantes
encodificadorOrdinal.fit(datosConTiposHabilidades)
#Hacemos la transformación
datosTiposHabilidadesNombres=encodificadorOrdinal.transform(datosConTiposHabilidades)
datosTiposHabilidadesNombres

,Name,Tier,HP,Attack,Defense,Special Attack,Special Defense,Speed,Next Evolution(s),Moves,Type 1,Type 2,Ability 1,Ability 2,Ability 3
Name,,,,,,,,,,,,,,,
Abomasnow,1,PU,90,92,75,92,85,60,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le...",Grass,Ice,Snow Warning,Soundproof,None
Abomasnow-Mega,2,NUBL,90,132,105,132,105,30,[],"['Ice Punch', 'Powder Snow', 'Leer', 'Razor Le...",Grass,Ice,Snow Warning,None,None
Abra,3,LC,25,20,15,105,55,90,['Kadabra'],"['Teleport', 'Ally Switch', 'Barrier', 'Encore...",‘Psychic’,None,Inner Focus,Magic Guard,Synchronize
Absol,4,PU,65,130,60,75,60,75,[],"['Perish Song', 'Future Sight', 'Me First', 'R...",Dark,None,Justified,Pressure,Super Luck
Absol-Mega,5,RUBL,65,150,60,115,60,115,[],"['Perish Song', 'Future Sight', 'Me First', 'R...",Dark,None,Magic Bounce,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zubat,914,LC,40,45,35,30,40,55,['Golbat'],"['Absorb', 'Leech Life', 'Supersonic', 'Astoni...",Flying,Poison,Infiltrator,Inner Focus,None
Zweilous,915,NaN,72,85,70,65,70,58,['Hydreigon'],"['Double Hit', 'Dragon Rage', 'Focus Energy', ...",Dark,Dragon,Hustle,None,None
Zygarde,916,OU,108,100,121,81,95,95,"['Zygarde-10%', 'Zygarde-Complete']","['Glare', 'Bulldoze', 'Dragon Breath', 'Bite',...",Dragon,Ground,Aura Break,Power Construct,None


La columna de "siguientes evoluciones" puede tener relevancia ya que los Pokémon que todavía pueden evolucionar pueden utilizar un objeto llamado Mineral Evolutivo, que aumenta sus defensas en un 50%. No nos importa tanto si el Pokémon puede evolucionar una vez, dos veces o si tiene diferentes posibles evoluciones; solo si es capaz de evolucionar. Por lo tanto, codificaremos esta columna de la siguiente manera: si el Pokémon tiene alguna evolución posible, le asignaremos el valor 1; en caso contrario, le asignaremos el valor 0. Para ello, vamos a modificar la columna aplicándole dos tratamientos. Si la fila correspondiente es "[]", pondremos "´No", y en caso contrario pondremos "Si". Después, aplicaremos una codificación One Hot, adecuada ya que sólo generará una columna adicional al poder tomar esta variable solo dos valores.

Además, hay algunos Pokémon que pueden tener formas alternativas (que no se consideran evoluciones, por lo que no permiten el uso del Mineral Evolutivo) y que en el dataset presentan una lista no vacía en la columna de "siguientes evoluciones". Como no hay ningún criterio para detectar estos casos, los vamos a tratar manualmente. Estos Pokémon son **Giratina**, **Greninja**, **Kyurem**, **Landorus**, **Oricorio**, **Rotom**, **Shaymin**, **Thundurus**, **Tornadus** y **Zygarde**. Pese a que tengan una lista vacía, estos Pokémon los vamos a codificar como que **no** tienen evolución.

In [24]:
class EvolutionCategorizer():
    # Constructor de la clase
    def __init__(self, columns=None): #Si recibimos algo
        self.columns = columns

    #Fit
    def fit(self, X):
        X=pd.DataFrame(X) #Por si recibiéramos un array de numpy
        if self.columns == None:
            self.columns=X.columns
        return self

    #Transform
    def transform(self, X):
        #Creamos una copia de X para no perder los datos originales
        Xaux=X.copy()
        #Accedemos a la columna de tipos, y rellenamos las columnas acorde
        for c in self.columns:
            Xaux.loc[:,c] = np.where(Xaux.loc[:,c]=="[]", "No", "Si")
            #Ahora, consideramos las excepciones consideradas arriba
            pokemonEquivocados = ["Giratina", "Greninja", "Kyurem", "Landorus", "Oriciorio", "Rotom", "Shaymin", "Thundurus", "Tornaous", "Zygarde"]
            for poke in pokemonEquivocados:
                Xaux.loc[poke,c]="No"
        return Xaux

    #set_params
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

#Una vez aplicamos el EvolutionCategorizer(), aplicamos el One Hot Encoding. Para ello, hacemos una PipeLine
pipe_Evol =  Pipeline([('Categorizador de Evoluciones', EvolutionCategorizer(columns=["Next Evolution(s)"])), 
                       ('One Hot Encoding', ce.OneHotEncoder(cols = ["Next Evolution(s)"], handle_unknown="ignore"))])

In [26]:
pd.options.display.max_rows = 1000
pipe_Evol.fit(datosTiposHabilidadesNombres)
datosTiposHabilidadesNombresEvol = pipe_evol.transform(datosTiposHabilidadesNombres)

TypeError: EvolutionCategorizer.fit() takes 2 positional arguments but 3 were given